### <font color="blue">import libraries</font>

In [60]:
import numpy as np
import os
import cv2
import random
import torch
import torchvision
from torchvision import models
from torchvision import datasets, transforms
from torchvision import transforms
import torchvision.transforms as transforms
import torchvision.transforms.transforms as T
from torch.utils.data import Dataset, DataLoader
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import json
from PIL import Image
import matplotlib



### <font color="blue">load and split train data</font>

In [61]:
transforms = T.Compose([
    T.Resize([256, 256]),
    T.ToTensor(),
])

trainset = datasets.ImageFolder("../input/plant-seedlings-classification/train", transform = transforms)
#dataloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=8)
#print(dataloader.dataset.class_to_idx)類別數
#print(dataloader.dataset[img][label])
#data_len=len(dataloader.dataset)

## Split
data_len=len(trainset)
train_num=int(data_len*4/5)
valid_num=int(data_len/5)
if (train_num + valid_num) < data_len:
    valid_num = valid_num + 1
generator = torch.Generator().manual_seed(42)
#train_set, valid_set = torch.utils.data.random_split(dataloader.dataset, [train_num, valid_num], generator)
train_set, valid_set = torch.utils.data.random_split(trainset, [train_num, valid_num], generator)
train_load = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=8)
valid_load = torch.utils.data.DataLoader(valid_set, batch_size=64, shuffle=False, num_workers=8)
#print(len(train_set))
#print(valid_set[500][1])

### <font color="blue"> ResNet50 model</font>

In [62]:
device = torch.device('cuda')
model = torchvision.models.resnet50(pretrained=True, progress=True)
for param in model.parameters():
    param.requires_grad = False
    #param.requires_grad = True
# Replace the last fully-connected layer
# Parameters of newly constructed modules have requires_grad=True by default
model.fc = nn.Linear(2048, 12)
model = model.to(device, non_blocking=True)
# Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)


### <font color="blue"> train and test </font>

In [63]:
def train(model, device, train_loader, valid_loader, optimizer, epoch):
    model.train()   # for Dropout, Batch Normalization, etc.
    train_len = len(train_loader.dataset)
    valid_len = len(valid_loader.dataset)
    for i in range(epoch):
        for inputs, targets in train_loader:
            inputs = inputs.to(device, non_blocking=True)
            #inputs=inputs.unsqueeze(0)
            targets = targets.to(device, non_blocking=True)

            # forward
            outputs = model(inputs)
            loss = F.cross_entropy(outputs, targets)
            
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_correct, train_loss = test(model, device, train_loader)
        valid_correct, valid_loss = test(model, device, valid_loader)
        print('epoch',(i+1), ':\n', '    train correct = ', (train_correct*100/train_len), '%, valid correct = ', (valid_correct*100/valid_len),
             '\n     train loss = ', train_loss, '%, valid loss = ', valid_loss)

def test(model, device, test_loader):
    model.eval()    # equals to model.train(False)
    correct = 0
    total_loss = 0
    testnum = len(test_loader.dataset)
    for inputs, targets in test_loader:
        inputs = inputs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        
        with torch.no_grad():
            # forward    
            outputs = model(inputs)
            
            # evaluation, e.g.
            loss = F.cross_entropy(outputs, targets)
            total_loss += loss.item()
            pred = outputs.argmax(dim=1, keepdim=True)
            correct += pred.eq(targets.view_as(pred)).sum().item()
    return correct, total_loss

In [64]:
train(model, device, train_load, valid_load, optimizer, 50)


epoch 1 :
     train correct =  65.73684210526316 %, valid correct =  60.8421052631579 
     train loss =  62.06756293773651 %, valid loss =  16.85419988632202
epoch 2 :
     train correct =  78.28947368421052 %, valid correct =  71.78947368421052 
     train loss =  43.81373971700668 %, valid loss =  13.072760581970215


KeyboardInterrupt: 